In [229]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [230]:
data = pd.read_csv('IPL Ball-by-Ball 2008-2020.csv')

In [231]:
df = pd.read_csv('IPL Matches 2008-2020.csv')

## Preprocessing

In [232]:
df.replace({'Rising Pune Supergiant': 'Rising Pune Supergiants','Delhi Daredevils':'Delhi Capitals', 'Deccan Chargers':'Sunrisers Hyderabad', 'Pune Warriors':'Rising Pune Supergiants'},inplace=True)
df.head(5)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [233]:
df["city"].fillna("Dubai", inplace = True)

# Batsman KPIs

1. Strike rate
2. Percentage of not out
3. Percentage of dot balls
4. Percentage of boundaries
5. Man of the match count
6. Percentage of 30+ runs scored in matches
7.average runs per match
8.strike rate in power play and deth over

<b>Calculating Total Runs</b>

In [234]:
no_of_runs=pd.DataFrame(data.groupby('batsman')['batsman_runs'].agg(sum)).reset_index().sort_values(by='batsman_runs',ascending=False).reset_index(drop=True)
no_of_runs.head(5)

,batsman,batsman_runs
0,V Kohli,5878
1,SK Raina,5368
2,DA Warner,5254
3,RG Sharma,5230
4,S Dhawan,5197


<b> Calculating number of ball</b>

In [235]:
no_of_balls=data.groupby('batsman')['ball'].count().reset_index().sort_values(by='ball',ascending=False).reset_index(drop=True)
no_of_balls.head(5)

,batsman,ball
0,V Kohli,4609
1,S Dhawan,4208
2,RG Sharma,4088
3,SK Raina,4041
4,DA Warner,3819


<b> Calculating number of bounderies</b>

In [236]:
boundries= data.loc[(data['batsman_runs'] == 4) | (data['batsman_runs'] == 6)]
no_of_balls_hitted_boundry=boundries.groupby('batsman')['ball'].count().reset_index().sort_values(by='ball',ascending=False).reset_index(drop=True)
no_of_balls_hitted_boundry.columns = ['batsman','boundaries']
no_of_balls_hitted_boundry.head(5)

,batsman,boundaries
0,CH Gayle,733
1,V Kohli,706
2,DA Warner,705
3,S Dhawan,700
4,SK Raina,687


<b> Calciulating number of tinmes dismissed</b>

In [237]:
wickets=data[data['is_wicket']==1]
batsman_out=wickets.groupby('batsman')['is_wicket'].count().reset_index().sort_values(by='is_wicket',ascending=False).reset_index(drop=True)
batsman_out.columns = ['batsman','dismissed']
batsman_out.head(5)

,batsman,dismissed
0,RG Sharma,177
1,RV Uthappa,169
2,V Kohli,163
3,SK Raina,160
4,S Dhawan,152


<b> Calculating total number of times dismissed</b>

In [238]:
dot_balls=data[(data['batsman_runs']==0)].groupby('batsman')['ball'].count().reset_index()
dot_balls.columns = ['batsman','dot_balls']
dot_balls.head(5)

,batsman,dot_balls
0,A Ashish Reddy,61
1,A Chandila,3
2,A Chopra,45
3,A Choudhary,4
4,A Dananjaya,2


<b> Calculating total number of times player has won man of the match</b>

In [239]:
no_man_of_the_match=df.groupby('player_of_match')['id'].count().reset_index().sort_values(by='id',ascending=False).reset_index(drop=True)
no_man_of_the_match.columns = ['batsman','man_of_match']
no_man_of_the_match.head(5)

,batsman,man_of_match
0,AB de Villiers,23
1,CH Gayle,22
2,RG Sharma,18
3,DA Warner,17
4,MS Dhoni,17


<b> Calculating total number of times scored 30+ </b>

In [240]:
total_score_in_every_match=data.groupby(['batsman','id'])['batsman_runs'].sum().reset_index()
score_above_thirty=total_score_in_every_match[total_score_in_every_match['batsman_runs']>=30].reset_index()
total_matches_above_thirty=score_above_thirty.groupby('batsman')['id'].count().reset_index()
total_matches_above_thirty.columns = ['batsman','match_30+']
total_matches_above_thirty.head(5)

,batsman,match_30+
0,A Ashish Reddy,2
1,A Mishra,2
2,A Symonds,14
3,AA Jhunjhunwala,2
4,AB Agarkar,1


<b> Total number of innings played</b>

In [241]:
total_matches_played=total_score_in_every_match.groupby('batsman')['id'].count().reset_index()
total_matches_played.columns = ['batsman','total_matches']
total_matches_played.head(5)

,batsman,total_matches
0,A Ashish Reddy,23
1,A Chandila,2
2,A Chopra,6
3,A Choudhary,3
4,A Dananjaya,1


In [242]:
#merging the data
batsmanDf = pd.merge(no_of_runs,no_of_balls,on='batsman')
batsmanDf = batsmanDf.merge(no_of_balls_hitted_boundry,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(batsman_out,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(dot_balls,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(no_man_of_the_match, left_on='batsman', right_on='batsman', how='left')
batsmanDf = batsmanDf.merge(total_matches_above_thirty, left_on='batsman', right_on='batsman', how='left')
batsmanDf = batsmanDf.merge(total_matches_played, left_on='batsman', right_on='batsman', how='left')

#filtered based on atleast 100 balls played
batsmanDf = batsmanDf[batsmanDf["ball"] >= 100]

batsmanDf.head(5)

,batsman,batsman_runs,ball,boundaries,dismissed,dot_balls,man_of_match,match_30+,total_matches
0,V Kohli,5878,4609,706.0,163.0,1625.0,13.0,84.0,184
1,SK Raina,5368,4041,687.0,160.0,1412.0,14.0,76.0,189
2,DA Warner,5254,3819,705.0,126.0,1429.0,17.0,76.0,142
3,RG Sharma,5230,4088,672.0,177.0,1532.0,18.0,73.0,194
4,S Dhawan,5197,4208,700.0,152.0,1613.0,8.0,72.0,175


<b>Calculating Strike Rate,Percentage not out,Percentage dot balls,Percentage bounderies,Percentage 30+ runs</b>

In [243]:
batsmanDf['strike_rate']=np.round(batsmanDf['batsman_runs']/batsmanDf['ball']*100,2)
batsmanDf['%age_not_out']=np.round(100-(batsmanDf['dismissed']/batsmanDf['total_matches'])*100,2)
batsmanDf['%age_dot_balls']=np.round((batsmanDf['dot_balls']/batsmanDf['ball'])*100,2)
batsmanDf['%age_boundaries']=np.round((batsmanDf['boundaries']/batsmanDf['ball'])*100,2)
batsmanDf['%30+runs']=np.round((batsmanDf['match_30+']/batsmanDf['total_matches'])*100,2)
batsmanDf['average_suns_per_match']=np.round((batsmanDf['batsman_runs']/batsmanDf['total_matches']),2)
batsmanDf.head(5)

,batsman,batsman_runs,ball,boundaries,dismissed,dot_balls,man_of_match,match_30+,total_matches,strike_rate,%age_not_out,%age_dot_balls,%age_boundaries,%30+runs,average_suns_per_match
0,V Kohli,5878,4609,706.0,163.0,1625.0,13.0,84.0,184,127.53,11.41,35.26,15.32,45.65,31.95
1,SK Raina,5368,4041,687.0,160.0,1412.0,14.0,76.0,189,132.84,15.34,34.94,17.00,40.21,28.40
2,DA Warner,5254,3819,705.0,126.0,1429.0,17.0,76.0,142,137.58,11.27,37.42,18.46,53.52,37.00
3,RG Sharma,5230,4088,672.0,177.0,1532.0,18.0,73.0,194,127.94,8.76,37.48,16.44,37.63,26.96
4,S Dhawan,5197,4208,700.0,152.0,1613.0,8.0,72.0,175,123.50,13.14,38.33,16.63,41.14,29.70


In [244]:
powerplay_balls=data[data['over']<6].groupby('batsman').sum().reset_index()
powerplay_balls=powerplay_balls[powerplay_balls['ball']>100].reset_index()
powerplay_balls['Strike_rate_in_powerplay']=((powerplay_balls['batsman_runs']/powerplay_balls['ball'])*100)
powerplay_balls.sort_values(by='Strike_rate_in_powerplay',ascending=False).reset_index()
del powerplay_balls['id']
del powerplay_balls['inning']
del powerplay_balls['over']
del powerplay_balls['ball']
del powerplay_balls['batsman_runs']
del powerplay_balls['extra_runs']
del powerplay_balls['is_wicket']
del powerplay_balls['non_boundary']
deathover_balls=data[data['over']>15].groupby('batsman').sum().reset_index()
deathover_balls=deathover_balls[deathover_balls['ball']>100].reset_index()
deathover_balls['Strike_rate_in_death_overs']=((deathover_balls['batsman_runs']/deathover_balls['ball'])*100)
del deathover_balls['id']
del deathover_balls['inning']
del deathover_balls['over']
del deathover_balls['ball']
del deathover_balls['batsman_runs']
del deathover_balls['extra_runs']
del deathover_balls['is_wicket']
del deathover_balls['non_boundary']
batsmanDf=pd.merge(batsmanDf,powerplay_balls,left_on='batsman',right_on='batsman',how='left')
batsmanDf=pd.merge(batsmanDf,deathover_balls,left_on='batsman',right_on='batsman',how='left')
del batsmanDf['index_x']
del batsmanDf['index_y']
del batsmanDf['total_runs_x']
del batsmanDf['total_runs_y']

batsmanDf.head(5)

,batsman,batsman_runs,ball,boundaries,dismissed,dot_balls,man_of_match,match_30+,total_matches,strike_rate,%age_not_out,%age_dot_balls,%age_boundaries,%30+runs,average_suns_per_match,Strike_rate_in_powerplay,Strike_rate_in_death_overs
0,V Kohli,5878,4609,706.0,163.0,1625.0,13.0,84.0,184,127.53,11.41,35.26,15.32,45.65,31.95,31.277213,58.103241
1,SK Raina,5368,4041,687.0,160.0,1412.0,14.0,76.0,189,132.84,15.34,34.94,17.00,40.21,28.40,33.721697,46.993318
2,DA Warner,5254,3819,705.0,126.0,1429.0,17.0,76.0,142,137.58,11.27,37.42,18.46,53.52,37.00,37.585854,55.555556
3,RG Sharma,5230,4088,672.0,177.0,1532.0,18.0,73.0,194,127.94,8.76,37.48,16.44,37.63,26.96,30.887926,53.886010
4,S Dhawan,5197,4208,700.0,152.0,1613.0,8.0,72.0,175,123.50,13.14,38.33,16.63,41.14,29.70,33.506856,43.721973


# Bowler KPIs

1. Economy
2. Strike Rate
3. Average Wickets per match
4. Percentage of Dot balls
5. Percentage Boundaries
6. Economy for Powerplay, Middle over, Death overs
7. Bowling Average
8. 4 Wicket Haul

<b>Calculating total number of balls and overs bowled by bowler</b>

In [245]:
Bowler_over= data[data["extra_runs"]==0][["ball","bowler"]]
Bowler_over= Bowler_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_over["No_of_Overs"]=(np.floor(Bowler_over["ball"]/6))
Bowler_over.head(5)

,bowler,ball,No_of_Overs
0,Harbhajan Singh,3325,554.0
1,PP Chawla,3218,536.0
2,R Ashwin,3186,531.0
3,A Mishra,3128,521.0
4,SL Malinga,2753,458.0


<b>Calculating total runs given by bowler</b>

In [246]:
Bowler_Runs= data.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_Runs.columns=["bowler","Total_runs_given"]
Bowler_over = Bowler_over[Bowler_over["ball"] >= 60]
Bowler_Runs.head(5)

,bowler,Total_runs_given
0,PP Chawla,4330
1,Harbhajan Singh,4038
2,A Mishra,3913
3,DJ Bravo,3869
4,R Ashwin,3756


<b>Total number of bounderies given</b>

In [247]:
Bowler_boundary = data.query('batsman_runs == 4 | batsman_runs == 6')
Bowler_boundary = Bowler_boundary.groupby("bowler")["batsman_runs"].count().reset_index().sort_values(by="batsman_runs",ascending=False).reset_index(drop=True)
Bowler_boundary.columns=["bowler","Total_boundaries_given"]
Bowler_boundary.head(5)

,bowler,Total_boundaries_given
0,UT Yadav,509
1,PP Chawla,497
2,DJ Bravo,443
3,P Kumar,442
4,Harbhajan Singh,431


<b>Calculating total number of wickets given</b>

In [248]:
Bowler_wicket = data[data["dismissal_kind"] != "run out"]
Bowler_wicket= Bowler_wicket.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_wicket.columns=["bowler","Wickets"]
Bowler_wicket.head(5)

,bowler,Wickets
0,SL Malinga,170
1,A Mishra,160
2,PP Chawla,156
3,DJ Bravo,153
4,Harbhajan Singh,150


<b>Calculating total number of dot balls by bowler</b>

In [249]:
Bowler_Dots = data[data["total_runs"] == 0]
Bowler_Dots= Bowler_Dots.groupby("bowler")["total_runs"].count().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_Dots.columns=["bowler","Dot_balls"]
Bowler_Dots.head(5)

,bowler,Dot_balls
0,Harbhajan Singh,1244
1,R Ashwin,1166
2,B Kumar,1155
3,SL Malinga,1144
4,PP Chawla,1137


<b>Total number of matches bowlers played </b>

In [250]:
Bowler_matches = data.groupby("bowler")["id"].nunique().reset_index().sort_values(by="id",ascending=False).reset_index(drop=True)
Bowler_matches.columns=["bowler","no_of_match"]
Bowler_matches.head(5)

,bowler,no_of_match
0,PP Chawla,163
1,Harbhajan Singh,157
2,RA Jadeja,155
3,R Ashwin,151
4,A Mishra,150


<b>Calculating Power play overs,economy and runs given</b>

In [251]:
Bowler_powerplay_wickets = data[data["over"] < 6]
Bowler_powerplay_wickets =  Bowler_powerplay_wickets.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_powerplay_wickets.columns=["bowler","Powerplay_runs"]

Bowler_powerplay_runs = data.query('dismissal_kind != "run out" & over < 6')
Bowler_powerplay_runs= Bowler_powerplay_runs.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_powerplay_runs.columns=["bowler","Powerplay_wickets"]

Bowler_powerplay_over = data.query('extra_runs == 0 & over < 6')
Bowler_powerplay_over= Bowler_powerplay_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_powerplay_over["Powerplay_over"]=np.round(Bowler_powerplay_over["ball"]/6,1)

Bowler_powerplay = pd.merge(Bowler_powerplay_runs,Bowler_powerplay_wickets,on="bowler")
Bowler_powerplay = pd.merge(Bowler_powerplay, Bowler_powerplay_over, on="bowler")
Bowler_powerplay["Powerplay_economy"] = np.round(Bowler_powerplay["Powerplay_runs"] / Bowler_powerplay["Powerplay_over"],1)

del Bowler_powerplay['ball']
Bowler_powerplay.head(5)

,bowler,Powerplay_wickets,Powerplay_runs,Powerplay_over,Powerplay_economy
0,Z Khan,53,1492,209.7,7.1
1,Sandeep Sharma,53,1447,201.2,7.2
2,B Kumar,48,1472,234.0,6.3
3,UT Yadav,45,1424,174.3,8.2
4,DS Kulkarni,44,1274,165.8,7.7


<b>Calculating Power play overs,economy and runs given</b>

In [252]:
Bowler_middle_wickets = data.query('over >= 6 & over <= 15')
Bowler_middle_wickets =  Bowler_middle_wickets.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_middle_wickets.columns=["bowler","Middle_runs"]

Bowler_middle_runs = data.query('dismissal_kind != "run out" & over >= 6 & over <= 15')
Bowler_middle_runs= Bowler_middle_runs.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_middle_runs.columns=["bowler","Middle_wickets"]

Bowler_middle_over = data.query('extra_runs == 0 & over >= 6 & over <= 15')
Bowler_middle_over= Bowler_middle_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_middle_over["Middle_over"]=np.round(Bowler_middle_over["ball"]/6,1)

Bowler_middle = pd.merge(Bowler_middle_runs,Bowler_middle_wickets,on="bowler")
Bowler_middle = pd.merge(Bowler_middle, Bowler_middle_over,on="bowler")
Bowler_middle["Middle_economy"] = np.round(Bowler_middle["Middle_runs"] / Bowler_middle["Middle_over"],1)

del Bowler_middle['ball']
Bowler_middle.head(5)

,bowler,Middle_wickets,Middle_runs,Middle_over,Middle_economy
0,A Mishra,130,3319,457.3,7.3
1,PP Chawla,112,3463,436.2,7.9
2,Harbhajan Singh,101,2904,404.0,7.2
3,YS Chahal,94,2065,271.2,7.6
4,RA Jadeja,84,2854,371.3,7.7


<b>Calculating Death overs,economy and runs given</b>

In [253]:
Bowler_death_wickets = data.query('over >= 16')
Bowler_death_wickets =  Bowler_death_wickets.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_death_wickets.columns=["bowler","Death_wickets"]

Bowler_death_runs = data.query('dismissal_kind != "run out" & over >= 16')
Bowler_death_runs= Bowler_death_runs.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_death_runs.columns=["bowler","Death_runs"]

Bowler_death_over = data.query('extra_runs == 0 & over >= 16')
Bowler_death_over= Bowler_death_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_death_over["Death_over"]=np.round(Bowler_death_over["ball"]/6,1)

Bowler_death = pd.merge(Bowler_death_wickets,Bowler_death_runs,on="bowler")
Bowler_death = pd.merge(Bowler_death, Bowler_death_over,on="bowler")
Bowler_death["Death_economy"] = np.round(Bowler_death["Death_runs"] / Bowler_death["Death_over"],1)

del Bowler_death['ball']
Bowler_death.head(5)

,bowler,Death_wickets,Death_runs,Death_over,Death_economy
0,DJ Bravo,1599,82,155.2,0.5
1,B Kumar,1267,66,129.2,0.5
2,SL Malinga,1252,90,147.0,0.6
3,JJ Bumrah,1208,53,127.5,0.4
4,UT Yadav,1072,42,95.7,0.4


<b>calculating number of times bowler has taken 4 wickets</b>

In [254]:
data_without_runouts = data[data["dismissal_kind"] != "run out"]
no_of_4_wickets=data_without_runouts.groupby(['bowler','id'])['is_wicket'].agg(sum).reset_index(name="wickets")
no_of_4_wickets.head(5)

,bowler,id,wickets
0,A Ashish Reddy,548329,0
1,A Ashish Reddy,548341,2
2,A Ashish Reddy,548346,1
3,A Ashish Reddy,548348,1
4,A Ashish Reddy,548352,1


In [255]:
no_of_4_wickets=no_of_4_wickets[no_of_4_wickets['wickets']>=4]
no_of_4_wickets=no_of_4_wickets.groupby('bowler').size().reset_index(name='4W')
no_of_4_wickets.head(5)

,bowler,4W
0,A Chandila,1
1,A Kumble,3
2,A Mishra,4
3,A Nehra,1
4,A Singh,1


In [256]:
#Merging the data
Bowler_details = Bowler_over.merge(Bowler_Runs,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_wicket,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_matches,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_boundary,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_Dots,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_powerplay,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_middle,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_death,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(no_of_4_wickets,left_on='bowler',right_on='bowler',how='left')
Bowler_details.head(5)

,bowler,ball,No_of_Overs,Total_runs_given,Wickets,no_of_match,Total_boundaries_given,Dot_balls,Powerplay_wickets,Powerplay_runs,...,Powerplay_economy,Middle_wickets,Middle_runs,Middle_over,Middle_economy,Death_wickets,Death_runs,Death_over,Death_economy,4W
0,Harbhajan Singh,3325,554.0,4038,150,157,431,1244,30.0,894.0,...,7.3,101.0,2904.0,404.0,7.2,240.0,19.0,28.2,0.7,2.0
1,PP Chawla,3218,536.0,4330,156,163,497,1137,18.0,445.0,...,7.6,112.0,3463.0,436.2,7.9,422.0,26.0,41.7,0.6,2.0
2,R Ashwin,3186,531.0,3756,138,151,361,1166,42.0,1003.0,...,6.7,83.0,2384.0,338.0,7.1,369.0,13.0,42.2,0.3,1.0
3,A Mishra,3128,521.0,3913,160,150,394,1125,6.0,164.0,...,7.6,130.0,3319.0,457.3,7.3,430.0,24.0,42.3,0.6,4.0
4,SL Malinga,2753,458.0,3486,170,122,400,1144,37.0,1210.0,...,6.7,43.0,1024.0,131.2,7.8,1252.0,90.0,147.0,0.6,7.0


<b>Calculating Economy,Strike Rate,Avg. wickets,% dot balls,% bounderies,bowling avg</b>

In [257]:
Bowler_details["Economy"] = np.round(Bowler_details["Total_runs_given"]/Bowler_details["No_of_Overs"],1)
Bowler_details["Strike Rate"] = np.round(Bowler_details["ball"]/Bowler_details["Wickets"],1)
Bowler_details["Avg. Wickets"] = np.round(Bowler_details["Wickets"]/Bowler_details["no_of_match"],1)
Bowler_details["%age of dots"] = np.round((Bowler_details["Dot_balls"]/Bowler_details["ball"]) * 100,1)
Bowler_details["%boundaries"] = np.round((Bowler_details["Total_boundaries_given"]/Bowler_details["ball"]) * 100,1)
Bowler_details["bowl_avg"] = np.round((Bowler_details["Total_runs_given"]/Bowler_details["Wickets"]),1)
Bowler_details.fillna(0.0)
Bowler_details.replace(np.nan, 0)
Bowler_details.head(5)

,bowler,ball,No_of_Overs,Total_runs_given,Wickets,no_of_match,Total_boundaries_given,Dot_balls,Powerplay_wickets,Powerplay_runs,...,Death_runs,Death_over,Death_economy,4W,Economy,Strike Rate,Avg. Wickets,%age of dots,%boundaries,bowl_avg
0,Harbhajan Singh,3325,554.0,4038,150,157,431,1244,30.0,894.0,...,19.0,28.2,0.7,2.0,7.3,22.2,1.0,37.4,13.0,26.9
1,PP Chawla,3218,536.0,4330,156,163,497,1137,18.0,445.0,...,26.0,41.7,0.6,2.0,8.1,20.6,1.0,35.3,15.4,27.8
2,R Ashwin,3186,531.0,3756,138,151,361,1166,42.0,1003.0,...,13.0,42.2,0.3,1.0,7.1,23.1,0.9,36.6,11.3,27.2
3,A Mishra,3128,521.0,3913,160,150,394,1125,6.0,164.0,...,24.0,42.3,0.6,4.0,7.5,19.6,1.1,36.0,12.6,24.5
4,SL Malinga,2753,458.0,3486,170,122,400,1144,37.0,1210.0,...,90.0,147.0,0.6,7.0,7.6,16.2,1.4,41.6,14.5,20.5


# Team KPIs

1. Percentage wins
2. Percentage wins by chasing
3. Percentage wins by defending
4. Percentage wins by tie
5. Percentage toss wins
6. Percentage home wins
7. Number of times qualified
8. Number of times toss and match winner

<b>Calculating number of matches for each team</b>

In [258]:
no_of_match_as_team1=df.groupby('team1').size().reset_index(name="no_of_match_as_team1")
no_of_match_as_team1=no_of_match_as_team1.rename(columns={"team1":"team_name"})
no_of_match_as_team1.head(5)

,team_name,no_of_match_as_team1
0,Chennai Super Kings,94
1,Delhi Capitals,102
2,Gujarat Lions,16
3,Kings XI Punjab,92
4,Kochi Tuskers Kerala,7


In [259]:
no_of_match_as_team2=df.groupby('team2').size().reset_index(name="no_of_match_as_team2")
no_of_match_as_team2=no_of_match_as_team2.rename(columns={"team2":"team_name"})
no_of_match_as_team2.head(5)

,team_name,no_of_match_as_team2
0,Chennai Super Kings,84
1,Delhi Capitals,92
2,Gujarat Lions,14
3,Kings XI Punjab,98
4,Kochi Tuskers Kerala,7


In [260]:
no_of_matches=pd.merge(no_of_match_as_team1,no_of_match_as_team2,on="team_name")
no_of_matches['total_no_of_matches_played']=no_of_matches['no_of_match_as_team1']+no_of_matches['no_of_match_as_team2']
no_of_matches.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played
0,Chennai Super Kings,94,84,178
1,Delhi Capitals,102,92,194
2,Gujarat Lions,16,14,30
3,Kings XI Punjab,92,98,190
4,Kochi Tuskers Kerala,7,7,14


<b>Insert the total number of wins for each teams</b>

In [261]:
no_of_wins=df.groupby('winner').size().reset_index(name="no_of_wins")
no_of_wins=no_of_wins.rename(columns={'winner':'team_name'})
no_of_wins.head(5)

,team_name,no_of_wins
0,Chennai Super Kings,106
1,Delhi Capitals,86
2,Gujarat Lions,13
3,Kings XI Punjab,88
4,Kochi Tuskers Kerala,6


In [262]:
#merging
teams_df=pd.merge(no_of_matches,no_of_wins,on='team_name')
teams_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins
0,Chennai Super Kings,94,84,178,106
1,Delhi Capitals,102,92,194,86
2,Gujarat Lions,16,14,30,13
3,Kings XI Punjab,92,98,190,88
4,Kochi Tuskers Kerala,7,7,14,6


<b>Calculating the number of wins by defending</b>

In [263]:
win_by_defending=df[df['result']=='runs'].groupby('winner').size().reset_index(name="wins_defend")
win_by_defending=win_by_defending.rename(columns={'winner':'team_name'})
win_by_defending.head(5)

,team_name,wins_defend
0,Chennai Super Kings,53
1,Delhi Capitals,34
2,Gujarat Lions,1
3,Kings XI Punjab,40
4,Kochi Tuskers Kerala,2


<b>Calculating total number of wins by chase</b>

In [264]:
win_by_chase=df[df['result']=='wickets'].groupby('winner').size().reset_index(name="wins_chase")
win_by_chase=win_by_chase.rename(columns={'winner':'team_name'})
win_by_chase.head(5)

,team_name,wins_chase
0,Chennai Super Kings,53
1,Delhi Capitals,50
2,Gujarat Lions,12
3,Kings XI Punjab,45
4,Kochi Tuskers Kerala,4


<b>Calculating toss wins for each team</b>

In [265]:
toss_wins=df.groupby('toss_winner').size().reset_index(name="toss_wins")
toss_wins=toss_wins.rename(columns={'toss_winner':'team_name'})
toss_wins.head(5)

,team_name,toss_wins
0,Chennai Super Kings,97
1,Delhi Capitals,100
2,Gujarat Lions,15
3,Kings XI Punjab,85
4,Kochi Tuskers Kerala,8


In [266]:
win_by_tie=df[df['result']=='tie'].groupby('winner').size().reset_index(name="wins_tie")
win_by_tie=win_by_tie.rename(columns={'winner':'team_name'})
win_by_tie.head(5)

,team_name,wins_tie
0,Delhi Capitals,2
1,Kings XI Punjab,3
2,Kolkata Knight Riders,1
3,Mumbai Indians,2
4,Rajasthan Royals,2


In [267]:
#merging
team_df = pd.merge(teams_df,win_by_chase,on="team_name")
team_df = pd.merge(team_df,win_by_defending,on="team_name")
team_df = team_df.merge(win_by_tie, left_on='team_name', right_on='team_name', how='left')
team_df = pd.merge(team_df, toss_wins, on ='team_name')
team_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97
1,Delhi Capitals,102,92,194,86,50,34,2.0,100
2,Gujarat Lions,16,14,30,13,12,1,NaN,15
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8


<b>Calculating percentage wins, percentage wins by chase,percentage wins by defend,percentage wins by tie and percentage toss</b>

In [268]:
teams_df['%win']=np.round((teams_df['no_of_wins']/teams_df['total_no_of_matches_played'])*100,1)
team_df['%win_chase']=np.round((team_df['wins_chase']/team_df['no_of_wins'])*100,2)
team_df['%win_defend']=np.round((team_df['wins_defend']/team_df['no_of_wins'])*100,2)
team_df['%win_tie']=np.round((team_df['wins_tie']/team_df['no_of_wins'])*100,1)
team_df['%toss_wins']=np.round((team_df['toss_wins']/team_df['total_no_of_matches_played'])*100,1)
team_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1


<b>Calculating Percentage win in homeground for each time</b>

In [269]:
data_citiy=pd.DataFrame(df.groupby('city'))
homegrounds={'Kolkata':'Kolkata Knight Riders','Bangalore':'Royal Challengers Bangalore','Bengaluru':'Royal Challengers Bangalore',
            'Chandigarh':'Kings XI Punjab','Chennai':'Chennai Super Kings','Delhi':'Delhi Capitals','Hyderabad':'Sunrisers Hyderabad',
            'Jaipur':'Rajasthan Royals','Rajkot':'Gujarat Lions','Kochi':'Kochi Tuskers Kerala','Kanpur':'Gujarat Lions',
            'Visakhapatnam':'Rising Pune Supergiants','Pune':'Rising Pune Supergiants','Cuttack':'Sunrisers Hyderabad',
            'Dharamshala':'Kings XI Punjab','Ahmedabad':'Rajasthan Royals','Mohali':'Kings XI Punjab','Mumbai':'Mumbai Indians'}

In [270]:
data_without_neutral=df[df['neutral_venue']!=1]
data_without_neutral=data_without_neutral[data_without_neutral['city']!='Dubai']
data_without_neutral['home_venue']=data_without_neutral['city'].map(homegrounds)
data_without_neutral.head(5)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,home_venue
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,Royal Challengers Bangalore
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,Kings XI Punjab
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,Delhi Capitals
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,Mumbai Indians
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,Kolkata Knight Riders


In [271]:
data_without_neutral['win']=np.where(data_without_neutral['winner'] == data_without_neutral['home_venue'], True, False)
data_without_neutral.head(5)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,home_venue,win
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,Royal Challengers Bangalore,False
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,Kings XI Punjab,False
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,Delhi Capitals,True
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,Mumbai Indians,False
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,Kolkata Knight Riders,True


In [272]:
no_of_wins_homeground=data_without_neutral[data_without_neutral['win']==True]
no_of_wins_homeground_per_team=no_of_wins_homeground.groupby('winner').size().reset_index(name="home_wins_count")
no_of_wins_homeground_per_team=no_of_wins_homeground_per_team.rename(columns={'winner':'team_name'})
no_of_wins_homeground_per_team.head(5)

,team_name,home_wins_count
0,Chennai Super Kings,40
1,Delhi Capitals,31
2,Gujarat Lions,5
3,Kings XI Punjab,30
4,Kochi Tuskers Kerala,2


In [273]:
total_home_matches_per_team=data_without_neutral.groupby('home_venue').size().reset_index(name="home_match_count")
total_home_matches_per_team=total_home_matches_per_team.rename(columns={'home_venue':'team_name'})

In [274]:
total_home_matches_per_team.head(5)

,team_name,home_match_count
0,Chennai Super Kings,57
1,Delhi Capitals,74
2,Gujarat Lions,14
3,Kings XI Punjab,56
4,Kochi Tuskers Kerala,5


In [275]:
home_match_data=total_home_matches_per_team.merge(no_of_wins_homeground_per_team,left_on='team_name',right_on='team_name', how='left')
home_match_data.head(5)

,team_name,home_match_count,home_wins_count
0,Chennai Super Kings,57,40
1,Delhi Capitals,74,31
2,Gujarat Lions,14,5
3,Kings XI Punjab,56,30
4,Kochi Tuskers Kerala,5,2


In [276]:
home_match_data['%home_win']=np.round((home_match_data['home_wins_count']/home_match_data['home_match_count'])*100,2)
home_match_data.head(5)

,team_name,home_match_count,home_wins_count,%home_win
0,Chennai Super Kings,57,40,70.18
1,Delhi Capitals,74,31,41.89
2,Gujarat Lions,14,5,35.71
3,Kings XI Punjab,56,30,53.57
4,Kochi Tuskers Kerala,5,2,40.00


In [277]:
del home_match_data['home_match_count']
del home_match_data['home_wins_count']
home_match_data.head(5)

,team_name,%home_win
0,Chennai Super Kings,70.18
1,Delhi Capitals,41.89
2,Gujarat Lions,35.71
3,Kings XI Punjab,53.57
4,Kochi Tuskers Kerala,40.00


In [278]:
team_df=team_df.merge(home_match_data, left_on='team_name', right_on='team_name', how='left')
team_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins,%home_win
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5,70.18
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5,41.89
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0,35.71
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7,53.57
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1,40.00


<b>Calculating the total number of times each team qualified</b>

In [279]:
#added year column in main data
df['year']=df['date'].map(lambda x:x[0:4])
df.head(5)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,year
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,2008
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,2008
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,2008
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,2008


In [280]:
year_wise_match_team1=df.groupby(['year','team1']).size().reset_index(name="no_of_match_team1")
year_wise_match_team1=year_wise_match_team1.rename(columns={'team1':'team_name'})
year_wise_match_team1.head(5)

,year,team_name,no_of_match_team1
0,2008,Chennai Super Kings,9
1,2008,Delhi Capitals,7
2,2008,Kings XI Punjab,7
3,2008,Kolkata Knight Riders,7
4,2008,Mumbai Indians,7


In [281]:
year_wise_match_team2=df.groupby(['year','team2']).size().reset_index(name="no_of_match_team2")
year_wise_match_team2=year_wise_match_team2.rename(columns={'team2':'team_name'})
year_wise_match_team2.head(5)

,year,team_name,no_of_match_team2
0,2008,Chennai Super Kings,7
1,2008,Delhi Capitals,7
2,2008,Kings XI Punjab,8
3,2008,Kolkata Knight Riders,6
4,2008,Mumbai Indians,7


In [282]:
year_wise_matches=pd.merge(year_wise_match_team1,year_wise_match_team2,on=['year','team_name'])
year_wise_matches['total_matches']=year_wise_matches['no_of_match_team1']+year_wise_matches['no_of_match_team2']
year_wise_matches.head(5)

,year,team_name,no_of_match_team1,no_of_match_team2,total_matches
0,2008,Chennai Super Kings,9,7,16
1,2008,Delhi Capitals,7,7,14
2,2008,Kings XI Punjab,7,8,15
3,2008,Kolkata Knight Riders,7,6,13
4,2008,Mumbai Indians,7,7,14


In [283]:
year_wise_qualified=year_wise_matches[year_wise_matches['total_matches']>14]
year_wise_qualified.head(5)

,year,team_name,no_of_match_team1,no_of_match_team2,total_matches
0,2008,Chennai Super Kings,9,7,16
2,2008,Kings XI Punjab,7,8,15
5,2008,Rajasthan Royals,7,9,16
9,2009,Delhi Capitals,9,6,15
13,2009,Sunrisers Hyderabad,10,6,16


In [284]:
no_of_time_team_qualified=year_wise_qualified.groupby('team_name').size().reset_index(name='toatal_no_of_time_qualified')
no_of_time_team_qualified.head(5)

,team_name,toatal_no_of_time_qualified
0,Chennai Super Kings,9
1,Delhi Capitals,5
2,Gujarat Lions,1
3,Kings XI Punjab,4
4,Kolkata Knight Riders,7


In [285]:
team_df=team_df.merge(no_of_time_team_qualified,left_on='team_name',right_on='team_name',how='left')
team_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins,%home_win,toatal_no_of_time_qualified
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5,70.18,9.0
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5,41.89,5.0
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0,35.71,1.0
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7,53.57,4.0
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1,40.00,NaN


In [286]:
toss_match_winner=df[df['toss_winner']==df['winner']]
toss_match_winner=toss_match_winner.groupby('winner').size().reset_index(name='toss_match_winner')
team_df=pd.merge(team_df,toss_match_winner,left_on='team_name',right_on='winner',how='left')
team_df

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins,%home_win,toatal_no_of_time_qualified,winner,toss_match_winner
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5,70.18,9.0,Chennai Super Kings,61
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5,41.89,5.0,Delhi Capitals,45
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0,35.71,1.0,Gujarat Lions,10
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7,53.57,4.0,Kings XI Punjab,36
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1,40.00,NaN,Kochi Tuskers Kerala,4
5,Kolkata Knight Riders,95,97,192,99,57,41,1.0,98,57.58,41.41,1.0,51.0,58.44,7.0,Kolkata Knight Riders,55
6,Mumbai Indians,97,106,203,120,56,62,2.0,106,46.67,51.67,1.7,52.2,52.48,9.0,Mumbai Indians,61
7,Rajasthan Royals,70,91,161,81,51,28,2.0,87,62.96,34.57,2.5,54.0,66.10,4.0,Rajasthan Royals,44
8,Rising Pune Supergiants,37,39,76,27,14,13,NaN,33,51.85,48.15,NaN,43.4,21.57,3.0,Rising Pune Supergiants,11
9,Royal Challengers Bangalore,108,87,195,91,51,38,2.0,87,56.04,41.76,2.2,44.6,46.25,7.0,Royal Challengers Bangalore,43


# Match KPIs

1. Percentage wins by chasing
2. Percentage wins by defending
3. Perceptage of tie matches
4. Percentage won the toss as well as match

<b>Removing matches tied</b>

In [287]:
#Excluding tie matches
total_matches = df[df['result'] != 'tie']
total_matches["result"].count()

799

<b>Wins by defending</b>

In [288]:
run_wins = total_matches[total_matches['result'] == 'runs']
run_wins['result'].count()

364

<b>Wins by chasing</b>

In [289]:
wicket_wins = total_matches[total_matches['result'] == 'wickets']
wicket_wins['result'].count()

435

<b>Percentage wins by chasing runs</b>

In [290]:
#Percentage wins by chasing runs
np.round((run_wins['result'].count()/total_matches["result"].count())*100,1)

45.6

<b>Percentage wins by defending</b>

In [291]:
#Percentage wins by defending
np.round((wicket_wins['result'].count()/total_matches["result"].count())*100,1)

54.4

<b>Perceptage of tie matches</b>

In [292]:
#Perceptage of tie matches
tie_matches = df[df['result'] == 'tie']
np.round(tie_matches['result'].count()/df["result"].count()*100,1)

1.6

<b>Percentage won the toss as well as match</b>

In [293]:
#Percentage won the toss as well as match
toss_winner = df[df['toss_winner'] == df['winner']]
np.round(toss_winner['result'].count()/df["result"].count()*100,1)

51.5

In [294]:
mergedTable = data.merge(df, left_on='id',right_on='id',how='left')
mergedTable.head(10)

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,...,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,year
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
5,335982,1,7,4,BB McCullum,RT Ponting,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
6,335982,1,7,5,RT Ponting,BB McCullum,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
7,335982,1,7,6,BB McCullum,RT Ponting,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
8,335982,1,8,1,BB McCullum,RT Ponting,JH Kallis,0,0,0,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
9,335982,1,8,2,BB McCullum,RT Ponting,JH Kallis,0,0,0,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008


In [295]:
year_data = mergedTable.groupby(['year'])['total_runs'].agg(sum).reset_index()
year_data

,year,total_runs
0,2008,17937
1,2009,16320
2,2010,18864
3,2011,21154
4,2012,22453
5,2013,22541
6,2014,18909
7,2015,18332
8,2016,18862
9,2017,18769


In [296]:
year_matches = mergedTable.groupby(['year'])['id'].nunique().reset_index()
year_matches.columns = ['year','matches']
year_matches

,year,matches
0,2008,58
1,2009,57
2,2010,60
3,2011,73
4,2012,74
5,2013,76
6,2014,60
7,2015,59
8,2016,60
9,2017,59


In [297]:
year_data = pd.merge(year_data,year_matches, on='year')
year_data['avg_runs'] = year_data['total_runs']/year_data['matches']
year_data

,year,total_runs,matches,avg_runs
0,2008,17937,58,309.258621
1,2009,16320,57,286.315789
2,2010,18864,60,314.400000
3,2011,21154,73,289.780822
4,2012,22453,74,303.418919
5,2013,22541,76,296.592105
6,2014,18909,60,315.150000
7,2015,18332,59,310.711864
8,2016,18862,60,314.366667
9,2017,18769,59,318.118644
